In [37]:
import boto3
import json
import sagemaker
from sagemaker import Session

runtime = boto3.client('sagemaker-runtime')

endpoint_name = "async-inference-endpoint"  # the name of the endpoint you've created

response = runtime.invoke_endpoint_async(EndpointName=endpoint_name,
                                  ContentType='application/json',  # or the relevant content type for your data
                                  Accept='application/json',
                                  InputLocation="s3://sagemaker-studio-mk6unewb9tb/inference_input/input_2.json"
                                  )

# SageMaker returns an identifier that allows you to retrieve results later.
print(response)


{'ResponseMetadata': {'RequestId': '9ceb8c77-34f5-44d5-9a76-cd9a93128398', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amzn-requestid': '9ceb8c77-34f5-44d5-9a76-cd9a93128398', 'x-amzn-sagemaker-outputlocation': 's3://sagemaker-studio-mk6unewb9tb/inference_output/dbb08720-2543-4d14-b244-5d169f6a1bd6.out', 'x-amzn-sagemaker-failurelocation': 's3://sagemaker-studio-mk6unewb9tb/async-endpoint-failures/pytorch-inference-2023-10-29-21-24-57-443-1698614698-f20e/dbb08720-2543-4d14-b244-5d169f6a1bd6-error.out', 'date': 'Sun, 29 Oct 2023 22:16:00 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'OutputLocation': 's3://sagemaker-studio-mk6unewb9tb/inference_output/dbb08720-2543-4d14-b244-5d169f6a1bd6.out', 'FailureLocation': 's3://sagemaker-studio-mk6unewb9tb/async-endpoint-failures/pytorch-inference-2023-10-29-21-24-57-443-1698614698-f20e/dbb08720-2543-4d14-b244-5d169f6a1bd6-error.out', 'InferenceId': '24ad1c57-1fc1-4ca8-8026-aefd4

In [33]:
import boto3


sm_client = boto3.client('sagemaker')

inference_id = response['InferenceId']
describe_response = sm_client.describe_async_inference_job(EndpointName=endpoint_name, InferenceId=inference_id)
describe_response

AttributeError: 'SageMaker' object has no attribute 'describe_async_inference_job'

In [17]:
import sagemaker

s3_output_uri = response['OutputLocation']  # assuming the response contains this key from earlier
bucket, key = sagemaker.s3.parse_s3_url(s3_output_uri)

s3_client = boto3.client('s3')
result_file = s3_client.get_object(Bucket=bucket, Key=key)
results = result_file['Body'].read().decode('utf-8')
print(results)

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [36]:
import boto3
import json

s3_client = boto3.client('s3')
bucket_name = 'sagemaker-studio-mk6unewb9tb'  # Replace with your S3 bucket name
object_key = 'inference_input/input_2.json'

# Your input payload
payload = {"data": "[0]", "output_length": 10}
input_json = json.dumps(payload)

# Upload the data to S3
s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=input_json)


{'ResponseMetadata': {'RequestId': 'G7EX01XFMVMDVNYV',
  'HostId': 'K7KPnYcvfVW5BfMj04wPYyy33mogGjCfPGVeBjx/6ya6AnppKAUyCUoWJfhHMqtBMzxDiSdfrvg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'K7KPnYcvfVW5BfMj04wPYyy33mogGjCfPGVeBjx/6ya6AnppKAUyCUoWJfhHMqtBMzxDiSdfrvg=',
   'x-amz-request-id': 'G7EX01XFMVMDVNYV',
   'date': 'Sun, 29 Oct 2023 22:15:52 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7fbcc03f8c295705910292b1ab486f61"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7fbcc03f8c295705910292b1ab486f61"',
 'ServerSideEncryption': 'AES256'}

In [35]:
import boto3

sagemaker_client = boto3.client('sagemaker')
endpoint_config_name = 'async-inference-endpoint'
response = sagemaker_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
response

{'EndpointConfigName': 'async-inference-endpoint',
 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:252201027045:endpoint-config/async-inference-endpoint',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'ModelName': 'pytorch-inference-2023-10-29-21-24-57-443',
   'InitialInstanceCount': 1,
   'InstanceType': 'ml.p3.8xlarge',
   'InitialVariantWeight': 1.0}],
 'CreationTime': datetime.datetime(2023, 10, 29, 14, 24, 58, 895000, tzinfo=tzlocal()),
 'AsyncInferenceConfig': {'OutputConfig': {'S3OutputPath': 's3://sagemaker-studio-mk6unewb9tb/inference_output/',
   'S3FailurePath': 's3://sagemaker-studio-mk6unewb9tb/async-endpoint-failures/pytorch-inference-2023-10-29-21-24-57-443-1698614698-f20e'}},
 'ResponseMetadata': {'RequestId': '8a00a683-dee1-4275-930a-a6fba475ccf5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8a00a683-dee1-4275-930a-a6fba475ccf5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '652',
   'date': 'Sun, 29 Oct 2023 

In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from sagemaker_inference import model_fn, input_fn, predict_fn


model_dict = model_fn('/Users/yifeiyan/micrograd-yifei')
output = predict_fn(("[0]", 4000), model_dict)
print(output)


YY 2023-11-02 10:44:23,712 [INFO]: YIFEIIIIII: model_fn - START
YY 2023-11-02 10:44:23,737 [INFO]: YIFEIIIIII: model_fn - END
YY 2023-11-02 10:44:23,738 [INFO]: YIFEIIIIII: predict_fn - START


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


YY 2023-11-02 10:44:34,027 [INFO]: YIFEIIIIII: predict_fn - END


	
"Capin Quirrell's black," have said. "Ask up one out ofth marb than I betweelves on.... the were Siriving passeep with
a boy hosto toward pairs words of a lroken this near. Sirius!"

Ron, though right soon, hurried, "I wouldn't be am the table.

"I say you know," Hermione. Finnon and brushings the Ron's twing;
minutely it couldn't lustely door. Nothing be his face, and the stopped,
crowding his judged, the face must a choposroard screctumble.

He told the whinderfamily that Dudley were people silenced. It was convisible, "'Be he found the uigh twelve them Dudley he'd they right better, foothers and Harry's straggled herst stop to reach. His eyeling dieved with
the word. Around Malfoy's lip into larger. He clearly been
their barding the milder bars sharp own to wazed to through the more windows vanistly; Air. And the could feet who rescramment room it be that thought hought him Together of Eye'd clarly right
of the doing convreson. He table to troubward, who found inter.
"You were see